## <center> Three-component seismograms</center>


## Step 1: Downloading data and plotting displacement seismograms

ObsPy functions used:

[*obspy.clients.fdsn - FDSN web service client for ObsPy*](https://docs.obspy.org/packages/obspy.clients.fdsn.html)

[*obspy.clients.fdsn.client.Client.get_stations*](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html#obspy.clients.fdsn.client.Client.get_stations) - downloads metadata (coordinates, sampling and timing information, responses)

[*obspy.clients.fdsn.client.Client.get_waveforms*](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html#obspy.clients.fdsn.client.Client.get_waveforms) - downloads data (waveforms)

[*obspy.core.stream.Stream.detrend*](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.detrend.html) - detrends waveforms

[*obspy.core.stream.Stream.remove_response*](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.remove_response.html) - corrects for instrument response

[*obspy.core.stream.Stream.taper*](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.taper.html) - tapers waveforms

[*obspy.core.stream.Stream.rotate*](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.rotate.html) - rotates seismogram components




In [ ]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np

#------------------------ importing ObsPy functions
import obspy
from obspy import read
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

#------------------ plotting mode
%matplotlib widget
#----------------------------



#------------------------ selecting an FDSN datacenter
client = Client('IRIS')

#---------------------------------------------------- defining start time for the data
tstart = UTCDateTime("2020-09-26T22:43:10.000")


#-------------------- defining duration of the downloaded time series in sec
t_duration = 30*60

#--------------- network station
ntw = "G"
sta = "SSB"

#--------------- downloading metadata
inv = client.get_stations(network=ntw,  starttime=tstart, endtime=tstart + t_duration, station=sta, channel="BH*", level="response")
#--------------- downloading waveforms
straw = client.get_waveforms(ntw, sta, "00", "BH*", tstart, tstart + t_duration)


print("network: ", stin[0].stats.network)
print("station: ", stin[0].stats.station)
print("discretization time step: ", stin[0].stats.delta)
print("number of samples: ", stin[0].stats.npts)


#--------- time exes series
dt = stin[0].stats.delta
npts = stin[0].stats.npts
time0 = dt*(np.linspace(1,npts,npts)-1)

dt = stin[1].stats.delta
npts = stin[1].stats.npts
time1 = dt*(np.linspace(1,npts,npts)-1)

dt = stin[2].stats.delta
npts = stin[2].stats.npts
time2 = dt*(np.linspace(1,npts,npts)-1)




straw.detrend('constant')
straw.detrend('linear')



#--------------- rotating seismograms to Z, N, and E components
straw.rotate(method="->ZNE", inventory=inv)

stin = straw.copy()


#------------------------------------------- plotting 3 displacement seismograms
fig, axs = plt.subplots(3, 1, sharex=True)
fig.subplots_adjust(hspace=0)

fig.suptitle(stin[0].stats.starttime.ctime() + '     ' + stin[0].stats.station + '   displacement')

axs[0].plot(time0, stin[0].data)
axs[0].set_title(stin[0].stats.channel,loc='right', y=0.8)

axs[1].plot(time1, stin[1].data)
axs[1].set_title(stin[1].stats.channel, loc='right', y=0.8)
axs[1].set_ylabel("counts")

axs[2].plot(time2, stin[2].data)
axs[2].set_title(stin[2].stats.channel, loc='right', y=0.8)
axs[2].set_xlabel('time (s)')

plt.show()
#-------------------

## Step 2: Filtering seismograms


In [ ]:
stin = straw.copy()


#stin.filter("bandpass", freqmin=0.03, freqmax=.15, corners=4, zerophase=True)
stin.filter("bandpass", freqmin=.6, freqmax=3, corners=4, zerophase=True)


#------------------------------------------- plotting 3 rotated displacement seismograms
fig, axs = plt.subplots(3, 1, sharex=True)
fig.subplots_adjust(hspace=0)

fig.suptitle(stin[0].stats.starttime.ctime() + '     ' + stin[0].stats.station + '  rotated displacement')

axs[0].plot(time0, stin[0].data)
axs[0].set_title(stin[0].stats.channel,loc='right', y=0.8)

axs[1].plot(time1, stin[1].data)
axs[1].set_title(stin[1].stats.channel, loc='right', y=0.8)
axs[1].set_ylabel("displacement (m)")

axs[2].plot(time2, stin[2].data)
axs[2].set_title(stin[2].stats.channel, loc='right', y=0.8)
axs[2].set_xlabel('time (s)')

plt.show()
#-------------------